In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
yolo = YOLO("YOLODataset/yolov8m.pt")

img = "valid/Documento_Nacional_Identidad_Frente_Versión_1_568760eb-65aa-4476-bf78-12468795e880.jpeg"

predict = yolo.predict(img, imgsz = 640, conf = 0.70)

# Cargar y mostrar la imagen
img = plt.imread(img)
fig, ax = plt.subplots()
ax.imshow(img)

# Obtener las coordenadas de la bbox
boxes = predict[0].boxes  # Acceder a las boxes del primer resultado
x1, y1, x2, y2 = boxes.xyxy[0].cpu().numpy()  # Obtener coordenadas de la primera bbox

# Crear el rectángulo
width = x2 - x1
height = y2 - y1
bbox = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')

# Agregar el rectángulo a la imagen
ax.add_patch(bbox)

plt.show()

In [ ]:
def process_directory(input_dir, output_dir, model_path):
    os.makedirs(output_dir, exist_ok=True)
    yolo = YOLO(model_path)
    
    valid_extensions = ('.jpg', '.jpeg', '.png')
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(valid_extensions)]
    
    for img_file in image_files:
        input_path = os.path.join(input_dir, img_file)
        output_path = os.path.join(output_dir, f"bbox_{img_file}")
            
        try:
            predict = yolo.predict(input_path, imgsz=640, conf=0.80)
            fig, ax = plt.subplots()
            img = plt.imread(input_path)    
            ax.imshow(img)
            
            if len(predict[0].boxes) > 0:
                boxes = predict[0].boxes
                # Obtener las clases y confianzas
                cls = boxes.cls.cpu().numpy()  # Índices de las clases
                conf = boxes.conf.cpu().numpy()  # Valores de confianza
                
                for i, box in enumerate(boxes.xyxy.cpu().numpy()):
                    x1, y1, x2, y2 = box
                    width = x2 - x1
                    height = y2 - y1
                    
                    # Obtener el nombre de la clase
                    class_id = int(cls[i])
                    class_name = predict[0].names[class_id]  # Obtiene el nombre de la clase
                    confidence = conf[i]
                    
                    # Crear el rectángulo
                    bbox = patches.Rectangle(
                        (x1, y1), width, height,
                        linewidth=2, edgecolor='r', facecolor='none'
                    )
                    ax.add_patch(bbox)
                    
                    # Añadir texto con clase y confianza
                    plt.text(x1, y1-10, f'{class_name} {confidence:.2f}', 
                            color='red', fontsize=8, bbox=dict(facecolor='white', alpha=0.7))
                    
                    # También imprimir en consola
                    print(f"Detección en {img_file}: Clase={class_name}, Confianza={confidence:.2f}")
            
            plt.axis('off')
            plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
            plt.close()
            
            print(f"Procesado: {img_file}")
            
        except Exception as e:
            print(f"Error procesando {img_file}: {str(e)}")
            continue

# Uso del script
input_directory = "dorso_test"
output_directory = input_directory + "_with_bbox"
model_path = "YOLODataset_seg/runs/segment/train2/weights/best.pt"

process_directory(input_directory, output_directory, model_path)